# Intermediate Slurm on Milton
### Delivered by WEHI Research Computing Platform

<pre>Edward Yang    Michael Milton    Julie Iskander</pre><br>

<img src="static/1200px-Slurm_logo.svg.png" alt="Slurm" width="100"/>
<img src="static/milton.png" alt="Milton Mascot" width="100"/>
<img src="static/WEHI_RGB_logo.png" alt="WEHI logo" width="200"/>

## [OPTIONAL SLIDE] Self Introductions!

### Me
* Civil Engineer by training
* HPC by interest
* Specialise in parallel programming (MPI, OpenMP, CUDA)
* I use HPC to simulate fluid and granular flows
* Historically flat-white person, but moving to long Macs

## How about you?
* name
* How you use Milton's HPC
* Your coffee/tea order

## Background
* We already ran an "intro to Slurm" workshop (recording on RCP website)
* More "advanced" features of Slurm were highly requested

### Target Audience
* You've submitted quite a few jobs via `sbatch`
* You're familiar with resource requests. Like:
    * using `--ntasks` and `--cpus-per-task`
    * using `--mem` and/or `--memory-per-cpu`
* You're wondering whether your jobs are utilizing resources efficiently
* You're wondering how to make your life easier when using `sbatch`

<div style="text-align: right">Background</div>

My goal for the session is to teach you how to:
* get the status of the cluster
* get information about your jobs
* make use of `sbatch` scripting features
* Run emabarrasingly parallel tasks with job arrays

## Agenda for Today

1. <p style="color: blue">Introduction & Housekeeping</p>


2. Lunch


3. Laying the groundwork: nodes, tasks and other Slurm terminology


4. Understanding your jobs and the job queue


5. Basic job profiling


6. Slurm scripting features


7. Embarrasingly parallel workflows with Slurm job arrays

## Introduction and Housekeeping

<div style="text-align: right">Introduction and Housekeeping 1</div>

### Format
Slides + live coding

Live coding will be on Milton, so make sure you're connected to WEHI's VPN or staff network, or use RAP: https://rap.wehi.edu.au

Please follow along to reinforce learning!

Questions:
* Put your hand up whenever you have a question or have an issue running things
* Questions in the chat are welcome and will be addressed by helpers

Material is available here:
* slides: https://wehi-researchcomputing.github.io/intermediate-slurm-workshop
* material: https://github.com/WEHI-ResearchComputing/intermediate-slurm-workshop
    * the base demo scripts are in `demo-script`. We will build on these during the workshop

<div style="text-align: right">Introduction and Housekeeping 2</div>

### Expected understanding of
* Understanding of "resources" concept<br>
    * CPUs
    * RAM/memory
    * Nodes
    * gres (GPUs)


* Have used job submission commands
    * `srun    # executes a command/script/binary across tasks`
    * `salloc  # allocates resources to be used (interactively and/or via srun)`
    * `sbatch  # submits a script for later execution on requested resources`


* awareness of resource request options
    * `--ntasks=             # "tasks" recognised by srun`
    * `--nodes=              # no. of nodes`
    * `--ntasks-per-node=    # tasks per node`
    * `--cpus-per-task=      # cpus per task`
    * `--mem=                # memory required for entire job`
    * `--mem-per-cpu=        # memory required for each CPU`
    * `--gres=               # "general resource" (i.e. GPUs)`
    * `--time=               # requested wall time`

# LUNCH

## Laying the Groundwork: Nodes, Tasks and Other Slurm Stuff

Reviewing cluster concepts and briefly explaining <strong>tasks</strong> and <strong>job step</strong> Slurm concepts

<div style="text-align: right">Nodes, Tasks and Other Slurm Stuff 1</div>

### What are Nodes?
Nodes are essentially standalone computers with their own CPU cores, RAM, local storage, and maybe GPUs. 
<br>
<img src="static/node-diagram.png" alt="Node diagram" width="400">

<div style="text-align: right">Nodes, Tasks and Other Slurm Stuff 2</div>

HPC clusters (or just clusters) will consist of multiple nodes connected together through a (sometimes fast) network. <br>
<img src="static/cluster-diagram.png" alt="Cluster diagram" width="400">
<img src="static/shared-storage-diagram.png" alt="Shared storage diagram" width="400">

<div style="text-align: right">Nodes, Tasks and Other Slurm Stuff 3</div>

Nodes cannot collaborate on problems unless they are running a programmed designed that way.

It's like clicking your mouse on your PC, and expecting the click to register on a colleague's PC. 

It's possible, but needs a special program/protocol to do so!

Biological sciences and statistics tend not to make use of multiple nodes to cooperate on a single problem.

Hence, we recommend passing `--nodes=1`.

<div style="text-align: right">Nodes, Tasks and Other Slurm Stuff 4</div>

### So why are tasks useful/important?

The Slurm task model was created with "traditional HPC" in mind
* `srun` creates `ntasks` instances of a program which coordinate using MPI
* Some applications are designed to use multiple cores per task (hybrid MPI-OpenMP) for performance.

Tasks are not as relevant in bioinformatics, but Slurm nevertheless uses tasks for accounting/profiling purposes. 

Therefore, it's useful to have an understanding of tasks in order to interpret some of Slurm's job accounting/profiling outputs.

<div style="text-align: right">Nodes, Tasks and Other Slurm Stuff 5</div>

### What are tasks?
Tasks are a collection of resources (CPU cores, GPUs) expected to perform the same "task", or used by a single program e.g., via threads, Python multiprocessing, or OpenMP.
<img src="static/tasks-diagram1.png" alt="Tasks diagram" width="400">

<div style="text-align: right">Nodes, Tasks and Other Slurm Stuff 6</div>

A task can only be given resources co-located on a node. <br>
Multiple tasks requested by `sbatch` or `salloc` can be spread across multiple nodes (unless `--nodes=` is specified).

For example, if we have two nodes with 4 CPU cores each:

requesting 1 task and 8 cpus-per-task won't work.

But requesting 2 tasks and 4 cpus-per-task will!

Most data science, statistics, bionformatics, health-science work will use `--ntasks=1`, and using `--cpus-per-task`. 

If you see/hear anything to do with "distributed" or MPI (e.g. distributed ML), you may want to change these options.

<div style="text-align: right">Nodes, Tasks and Other Slurm Stuff 7</div>

### A review of launching jobs (`srun` vs `sbatch` and `salloc`)

TL;DR:
* `sbatch` requests resources for use with a script
* `salloc` requests resources to be used interactively
* `srun` runs programs/scripts using resources requested by `sbatch` and `salloc`
    * When run "inside" a job, `srun` will use the resources requested in the parent request
    * When run "outside" a job, `srun` will need to be passed the resource request e.g. `--ntasks`

`srun` will execute `ntasks` instances of the same command/script/program. e.g.,

`srun echo hello world` will run `echo` equal to the number of tasks requested.

## Monitoring Your Jobs and the Job Queue

Using Slurm and system tools to understand what your jobs are doing

<div style="text-align: right">Monitoring Your Jobs and the Job Queue 1</div>
Slurm has lots of data on your jobs and the cluster!

Primary utilities discussed in this section:
* `squeue`    _Live_ Job queue data
* `scontrol`  _Live_ Singular job data
* `sinfo`     _Live_ Cluster data

This section show you how to get more detailed information about:
* the queue,
* other jobs in the queue, and
* the state/business of the cluster.

<div style="text-align: right">Monitoring Your Jobs and the Job Queue 2</div>

### Building on the basics: `squeue`

`squeue` shows everyone's job in the queue (passing `-u <username>`) shows only `<username>`'s jobs.

In [4]:
squeue | head -n 5

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
           8516030      gpuq interact bollands  R    1:30:11      1 gpu-p100-n01
           8515707      gpuq cryospar cryospar  R    3:04:59      1 gpu-p100-n01
           8511988 interacti sys/dash    yan.a  R   20:15:53      1 sml-n03
           8516092 interacti     work jackson.  R    1:21:42      1 sml-n01


<div style="text-align: right">Monitoring Your Jobs and the Job Queue 3</div>
Getting a bit more: `squeue --long` makes things more legible adds the "time_limit" column.

In [1]:
squeue --long | head -n 5

Thu Nov 03 12:57:40 2022
             JOBID PARTITION     NAME     USER    STATE       TIME TIME_LIMI  NODES NODELIST(REASON)
           8649808 gpuq_larg   AF2.2g iskander  RUNNING    4:34:07 5-00:00:00      1 gpu-a100-n02
           8649805 gpuq_larg   AF2.2g iskander  RUNNING 1-03:47:14 5-00:00:00      1 gpu-a100-n01
           8664606 interacti sys/dash     wu.y  RUNNING    2:12:59   8:00:00      1 med-n01


<div style="text-align: right">Monitoring Your Jobs and the Job Queue 4</div>

But what if we want _even more_ information?

We have to make use of the formatting options!

```
$ squeue --Format field1,field2,...
```

OR use the environment variable `SQUEUE_FORMAT2`. Useful fields:

| Resources related | Time related | Scheduling   |
| :---              | :---         | :---         |
| `NumCPUs`         | `starttime`  | `JobId`      |
| `NumNodes`        | `submittime` | `name`       |
| `minmemory`       | `pendingtime`| `partition`  |
| `tres-alloc`      | `timelimit`  | `priority`   |
| `minmemory`       | `timeleft`   | `reasonlist` |
|                   | `timeused`   | `workdir`    |
|                   |              | `state`      |

You can always use `man squeue` to see the entire list of options.

So you don't have to type out the fields, I recommend aliasing the the command with your fields of choice in `~/.bashrc` e.g.

<div style="text-align: right">Monitoring Your Jobs and the Job Queue 5</div>

In [26]:
alias sqv="squeue --Format=jobid:8,name:6' ',partition:10' ',statecompact:3,tres-alloc:60,timelimit:12,timeleft:12"
sqv | head -n 5

JOBID   NAME   PARTITION  ST TRES_ALLOC                                                  TIME_LIMIT  TIME_LEFT   
8517002 R      bigmem     R  cpu=22,mem=88G,node=1,billing=720984                        1-00:00:00  23:35:18    
8516030 intera gpuq       R  cpu=2,mem=20G,node=1,billing=44,gres/gpu=1,gres/gpu:p100=1  8:00:00     4:43:00     
8515707 cryosp gpuq       R  cpu=8,mem=17G,node=1,billing=44,gres/gpu=1,gres/gpu:p100=1  2-00:00:00  1-19:08:12  
8511988 sys/da interactiv R  cpu=8,mem=16G,node=1,billing=112                            1-00:00:00  1:57:18     


In [28]:
sqv -u bedo.j | head -n 5

JOBID   NAME   PARTITION  ST TRES_ALLOC                                                  TIME_LIMIT  TIME_LEFT   
8516851 bionix regular    PD cpu=24,mem=90G,node=1,billing=204                           2-00:00:00  2-00:00:00  
8516850 bionix regular    PD cpu=24,mem=90G,node=1,billing=204                           2-00:00:00  2-00:00:00  
8516849 bionix regular    PD cpu=24,mem=90G,node=1,billing=204                           2-00:00:00  2-00:00:00  
8516848 bionix regular    PD cpu=24,mem=90G,node=1,billing=204                           2-00:00:00  2-00:00:00  


<div style="text-align: right">Monitoring Your Jobs and the Job Queue 6</div>

### Getting detailed information of your running/pending job

`scontrol show job <jobid>`

Useful if you care only about a specific job.

It's very useful when debugging jobs.

A lot of information without needing lots of input.

In [30]:
scontrol show job 8516360

JobId=8516360 JobName=Extr16S23S
   UserId=woodruff.c(2317) GroupId=allstaff(10908) MCS_label=N/A
   Priority=324 Nice=0 Account=wehi QOS=normal
   JobState=RUNNING Reason=None Dependency=(null)
   Requeue=1 Restarts=0 BatchFlag=1 Reboot=0 ExitCode=0:0
   RunTime=00:21:53 TimeLimit=2-00:00:00 TimeMin=N/A
   SubmitTime=2022-10-20T11:37:49 EligibleTime=2022-10-20T11:37:49
   AccrueTime=2022-10-20T11:37:49
   StartTime=2022-10-20T14:28:03 EndTime=2022-10-22T14:28:03 Deadline=N/A
   PreemptEligibleTime=2022-10-20T14:28:03 PreemptTime=None
   SuspendTime=None SecsPreSuspend=0 LastSchedEval=2022-10-20T14:28:03 Scheduler=Main
   Partition=regular AllocNode:Sid=vc7-shared:12938
   ReqNodeList=(null) ExcNodeList=(null)
   NodeList=med-n24
   BatchHost=med-n24
   NumNodes=1 NumCPUs=32 NumTasks=32 CPUs/Task=1 ReqB:S:C:T=0:0:*:*
   TRES=cpu=32,mem=48G,node=1,billing=128
   Socks/Node=* NtasksPerN:B:S:C=32:0:*:* CoreSpec=*
   MinCPUsNode=32 MinMemoryNode=48G MinTmpDiskNode=0
   Features=(null) Dela

<div style="text-align: right"> Monitoring Your jobs and the Job Queue 7</div>

### Monitoring the cluster

Being able to understand the state of the cluster, can help understand why your job might be waiting.

Or, you can use the information to your advantage to reduce wait times.

To view the state of the cluster, we're going to use the `sinfo` command.

In [31]:
sinfo

PARTITION        AVAIL  TIMELIMIT  NODES  STATE NODELIST
interactive         up 1-00:00:00      4    mix med-n03,sml-n[01-03]
interactive         up 1-00:00:00      1  alloc med-n02
interactive         up 1-00:00:00      1   idle med-n01
regular*            up 2-00:00:00     42    mix lrg-n[02-03],med-n[03-05,07-09,12-13,18,20-23,25-27,29-30],sml-n[02-20,22-24]
regular*            up 2-00:00:00     13  alloc lrg-n04,med-n[02,06,10-11,14-17,19,24,28],sml-n21
long                up 14-00:00:0     40    mix med-n[03-05,07-09,12-13,18,20-23,25-27,29-30],sml-n[02-20,22-24]
long                up 14-00:00:0     12  alloc med-n[02,06,10-11,14-17,19,24,28],sml-n21
bigmem              up 2-00:00:00      3    mix lrg-n02,med-n[03-04]
bigmem              up 2-00:00:00      1  alloc med-n02
bigmem              up 2-00:00:00      1   idle lrg-n01
gpuq                up 2-00:00:00      1    mix gpu-p100-n01
gpuq                up 2-00:00:00     11   idle gpu-a30-n[01-07],gpu-p100-n[02-05]
gpuq_inter

<div style="text-align: right">Monitoring Your Jobs and the Job Queue 8</div>
Like `squeue`, we can augment `sinfo`'s behaviour with options.

A very useful option is to use the `-N` (N for nodes) option.

In [33]:
sinfo -N | head -n 5

NODELIST      NODES        PARTITION STATE 
gpu-a10-n01       1 gpuq_interactive mix   
gpu-a30-n01       1             gpuq idle  
gpu-a30-n02       1             gpuq idle  
gpu-a30-n03       1             gpuq idle  


And now the data is now node-oriented instead of partition oriented!

<div style="text-align: right">Monitoring Your Jobs and the Job Queue 9</div>
But just knowing whether nodes are "idle", "mixed", or "allocated" is not the _most useful_ information.

We can add detail with formatting options as well.

| CPU | memory | gres (GPU) | node state | time |
| :---| :--- | :--- | :--- | :--- |
| `CPUsState` | `FreeMem` | `GresUsed` | `StateCompact` | `Time` |
| | `AllocMem` | `Gres` | | |
| | `Memory` | | | |

* CPUs occupied/available/total: CPUsState
* memory occupied/available/total: FreeMem, AllocMem, Memory
* gres (GPU) occupied/available: GresUsed, Gres
* State of the node (e.g. whether the node is down): StateCompact
* Max time: Time

In [13]:
sinfo -NO nodelist:11' ',partition:10' ',cpusstate:13' ',freemem:8' ',memory:8' ',gresused,gres:11,statecompact:8,time | head -n 5

NODELIST    PARTITION  CPUS(A/I/O/T) FREE_MEM MEMORY   GRES_USED           GRES       STATE   TIMELIMIT           
gpu-a10-n01 gpuq_inter 0/48/0/48     163914   257417   gpu:A10:0(IDX:N/A)  gpu:A10:4  idle    12:00:00            
gpu-a30-n01 gpuq       0/96/0/96     450325   511362   gpu:A30:0(IDX:N/A)  gpu:A30:4  idle    2-00:00:00          
gpu-a30-n02 gpuq       0/96/0/96     436435   511362   gpu:A30:0(IDX:N/A)  gpu:A30:4  idle    2-00:00:00          
gpu-a30-n03 gpuq       0/96/0/96     497816   511362   gpu:A30:0(IDX:N/A)  gpu:A30:4  idle    2-00:00:00          


<div style='text-align: right'>Monitoring Your Jobs and the Job Queue 10</div>

For newer Slurm versions, the GUI tool `sview` is an option.

It's functionality is currently a little limited and not customizable.

Requires and X11 server running on your computer (Windows: MobaXTerm, Mac: XQuartz).

In [36]:
sview


(sview:4266): Gtk-WARNING **: 16:00:18.949: cannot open display: 


: 1

## Basic Job Monitoring and Profiling

Using command-line tools to obtain visibility into how your job is performing.

<div style='text-align: right'>Basic Job Monitoring and Profiling 1</div>

This section will look at using command-line tools to obtain visibility into how your job is performing.

| type of data | Live | Historical |
| --- | --- | --- |
| <strong>good for</strong> | debugging | debugging |
| | evaluating utilization | profiling |
| <strong>drawbacks</strong> | uses system tools, so requires some system understanding | Only provides data when jobs are completed |

We will look at:
* `htop` for _Live_ Process activity on nodes
* `nvidia-smi` for _Live_ GPU activity on nodes
* `seff` for _Historical_ job CPU and memory usage data
* `dcgmstats` for _Historical_ job GPU usage data
* `sacct` for _Historical_ job data

<div style='text-align: right'>Basic Job Monitoring and Profiling 2</div>

### Live monitoring of jobs

Slurm can't provide accurate "live" data about jobs' activities

System tools must be used instead.

This requires matching jobs to processes on a node with `squeue` and `ssh`.

A bit of system understanding is also needed.

<div style='text-align: right'>Basic Job Monitoring and Profiling 3</div>

#### Live monitoring: CPU, memory, IO activity
`htop` is a utility often installed on HPC clusters for monitoring processes.

It can be used to look at the CPU, memory, and IO utilization of a running process. 

It's not a Slurm tool, but is nevertheless very useful in monitoring jobs' activity and diagnosing issues.

To show only your processes, execute `htop -u $USER`

<div style='text-align: right'>Basic Job Monitoring and Profiling 4</div>

`htop` shows the individual CPU core utilization on the top, followed by memory utilization and some misc. information.

The bottom panel shows the process information

Relevant Headings:
* USER: User that owns the process
* PID: Process ID
* %CPU: % of a single core that a process is using e.g. 400% means process is using 4 cores
* %MEM: % of node's total RAM that process is using
* VSZ: "Virtual" memory (bytes) - the memory a process "thinks" it's using
* RSS: "Resident" memory (bytes) - the actual physical memory a process is using

<div style='text-align: right'>Basic Job Monitoring and Profiling 5</div>
You can organize the process information into "trees" by pressing `F5`

You can add IO information by
1. Press `F2` (Setup)
2. Press down three times to move the cursor to "Columns" and press right twice
3. The cursor should now be in "Available Columns". Scroll down to `IO_READ_RATE` and press enter
4. Scroll down to `IO_WRITE_RATE` and press enter
5. Press `F10` to exit. 
You should now be able to see read/write rates for processes that you have permissions for.

<strong>Tips</strong>: 
* `htop` configurations are saved in `~/.config/htop`. Delete this folder to reset your `htop` conifguration.
* `ps` and `pidstat` are useful alternatives which can be incorporated into scripts.
* Some systems may not have `htop` installed, in which case `top` can be used instead

<div style='text-align: right'>Basic Job Monitoring and Profiling 6</div>

#### Live monitoring: GPU activity

To monitor activity of Milton's NVIDIA GPUs, we must rely on NVIDIA's `nvidia-smi` tool.

`nvidia-smi` shows information about the memory and compute utilization, process allocation and other details. 

Like `htop`, `nvidia-smi` only provides information on processes running on a GPU. If your job is occupying an entire node and all its GPUs, it should be straightforward to determine which GPUs you've been allocated.

But if your job is sharing a node with other jobs, you might not know straight away which GPU your job has been allocated. You can determine this by
* inferring by the command being run on a GPU, or
* using `squeue` with extra formatting options as discussed previously.

<strong>Note</strong>:
This tool is available only on GPU nodes where the CUDA drivers are installed, so you must `ssh` to a `gpu` node to try it.

<strong>Tip</strong>: Combine `nvidia-smi` with `watch` to automatically update the output.

<div style='text-align: right'>Basic Job Monitoring and Profiling 7</div>

### Historical monitoring of jobs
Slurm tools and plugins are generally easier to use because they provide information on a per-job basis, meaning there's no need to match processes with jobs like previously discussed.

<strong>Tips</strong>: _generally_, results are more reliable when executing commands with `srun`.

<div style='text-align: right'>Basic Job Monitoring and Profiling 8</div>

#### Historical data: CPU and memory utilization
The `seff` command shows the memory and CPU utilization of a job that has <strong>ended</strong>`

In [35]:
seff 8665813

Job ID: 8665813
Cluster: milton
User/Group: yang.e/allstaff
State: COMPLETED (exit code 0)
Nodes: 1
Cores per node: 4
CPU Utilized: 00:09:04
CPU Efficiency: 99.27% of 00:09:08 core-walltime
Job Wall-clock time: 00:02:17
Memory Utilized: 1.95 GB (estimated maximum)
Memory Efficiency: 48.83% of 4.00 GB (1.00 GB/core)


<div style='text-align: right'>Basic Job Monitoring and Profiling 9</div>

`sacct` is a general job history querying command-line tool that can provide lots of information about your _past_ jobs.

<strong>Note</strong>: `sacct` data can take a few minutes to be updated, so is best for jobs that have finished.

The following `sacct` command shows your job data for jobs since 1st Nov:
* Job steps' ID and name
* Requested resources
* Elapsed time
* The quantity of data written and read
* The quantity of virtual and resident memory used

Note that the IO and memory values shown will be for the highest use <strong>task</strong>.

<div style='text-align: right'>Basic Job Monitoring and Profiling 10</div>

In [21]:
sacct -S 2022-11-01 -o jobid%14' ',jobname,ncpus%5' ',nodelist,elapsed,state,maxdiskread,maxdiskwrite,maxvmsize,maxrss | head -n5

         JobID    JobName NCPUS        NodeList    Elapsed      State  MaxDiskRead MaxDiskWrite  MaxVMSize     MaxRSS 
-------------- ---------- ----- --------------- ---------- ---------- ------------ ------------ ---------- ---------- 
       8664599 sys/dashb+     2         sml-n01 1-00:00:22    TIMEOUT                                                 
 8664599.batch      batch     2         sml-n01 1-00:00:23  CANCELLED      102.64M       15.11M   1760920K     99812K 
8664599.extern     extern     2         sml-n01 1-00:00:22  COMPLETED        0.00M            0    146612K        68K 


<div style='text-align: right'>Basic Job Monitoring and Profiling 11</div>

### A short aside on job _steps_
Slurm breaks jobs into steps. Jobs will have steps:
* `.extern`: work done not part of the job i.e. overhead
* `.<index>`: work done with `srun`
* `.batch`: work inside an `sbatch` script, but not executed by `srun`
* `.interactive`: work done inside an interactive `salloc` session, but not executed by `srun`.



<div style='text-align: right'>Basic Job Monitoring and Profiling 12</div>

#### Historical data: GPU activity
By default, Slurm doesn't have the ability to produce stats on GPU usage.

WEHI's ITS have implemented the `dcgmstats` NVIDIA Slurm plugin which can produce these summary stats.

To use this plugin, pass the `--comment=dcgmstats` option to `srun`, `salloc`, or `sbatch`.

If your job requested at least one GPU, an extra output file will be generated in the working directory called `dcgm-stats-<jobid>.out`. The output file will contain a table for each GPU requested by the job.

<div style='text-align: right'>Basic Job Monitoring and Profiling 13</div>

### Summary
* live monitoring:
    * `htop` for CPU, memory, and IO data (requires configuration)
    * `nvidia-smi` for GPU activity
    * both require matching jobs to hardware and running processes
* historical monitoring:
    * `seff` command for simple CPU and memory utilization data for one job
    * `sacct` command for memory and IO data for multiple past jobs
    * `dcgmstats` Slurm plugin for GPU stats for a single Slurm job

## Sbatch Scripting Features
Taking advantage of lesser-known options and environment features to make life easier

<div style='text-align: right'>Sbatch Scripting Features 1</div>
Sbatch scripts have a lot of nice features that extend beyond requesting resources. This section will look at some of these useful features which you can use in your workflows.

This section will look at:
* getting email notifications
* changing `stdout` and `stderr` files
* controlling how jobs depend on each other
* making use of job environments and interpreters (e.g. python or R)
* submitting `sbatch` scripts without a script

We're going to start with a simple R script submitted by wrapper sbatch script:

<div style='text-align: right'>Sbatch Scripting Features 2</div>

```r
## matmul.rscript
# multiplies two matrices together and prints how long it takes.

print("starting the matmul R script!")
nrows = 1e3
print(paste0("elem: ", nrows, "*", nrows, " = ", nrows*nrows))

# generating matrices
M <- matrix(rnorm(nrows*nrows),nrow=nrows)
N <- matrix(rnorm(nrows*nrows),nrow=nrows)

# start matmul
start.time <- Sys.time()
invisible(M %*% N)
end.time <- Sys.time()

# Getting final time and writing to stdout
elapsed.time <- difftime(time1=end.time, time2=start.time, units="secs")
print(elapsed.time)
```

In [23]:
#!/bin/bash
## submit-matmul.sh
# Example sbatch script running Rscript
# Does a matmul
# rev0

#SBATCH --mem=8G
#SBATCH --cpus-per-task=2
#SBATCH --time=1-
#SBATCH --nodes=1
#SBATCH --ntasks=1

# loading module for R
module load R/openBLAS/4.2.1

Rscript matmul.rscript

[1] "starting the matmul R script!"
[1] "elem: 1000*1000 = 1e+06"
Time difference of 0.06260109 secs


<div style='text-align: right'>Sbatch Scripting Features 4</div>

### Email notifications
Getting notifications about the status of your Slurm jobs remove the need to `ssh` onto Milton and running `squeue` to get the status of your jobs.

Instead, it will notify you when your job state has changed e.g. when it has started or ended.

To enable this behaviour, add the following options to your job scripts:
```
--mail-user=me@gmail.com
--mail-type=ALL
```
This sends emails to `me@gmail.com` when the job state changes. 

If you only want to know when your job goes through certain states, e.g. if it fails or is pre-empted but not when it starts or finishes:
* BEGIN: job starts
* END: job finishes successfully
* FAIL: job fails
* TIME_LIMIT: job reaches time limit
* TIME_LIMIT_50/80/90: job reaches 50%/80%/90% of time limit

<div style='text-align: right'>Sbatch Scripting Features 5</div>

```r
#!/bin/bash
# Example sbatch script running Rscript
# Does a matmul
# rev1 - email notifications

#SBATCH --mem=8G
#SBATCH --cpus-per-task=2
#SBATCH --time=1-
#SBATCH --nodes=1
#SBATCH --ntasks=1
#SBATCH --mail-user=yang.e@wehi.edu.au
#SBATCH --mail-type=ALL

# loading module for R
module load R/openBLAS/4.2.1

Rscript matmul.rscript
```

<div style='text-align: right'>Sbatch Scripting Features 6</div>

### `sbatch` without a script
In some cases, one may wish to submit singular commands to the scheduler. `srun` and `salloc` can do this, but they need a terminal attached i.e., if you close your terminal with the `srun` or `salloc` session, then the job fails.

the `sbatch --wrap` option allows you to submit a singular command instead of an entire script.

This can be useful for testing, or implementing `sbatch` inside a script that manages your workflow.

Note that `sbatch --wrap` infers which interpreter to use from your active environment.

<div style='text-align: right'>Sbatch Scripting Features 7</div>

### a short aside on `stdout` and `stderr`
Linux uses has two main "channels" to send output messages to. One is "stdout" (standard out), and the other is "stderr" (standard error).

If you have ever used the `|` `>` or `>>` shell scripting features, then you've _redirected_ `stdout` somewhere else e.g., to another command, a file, or the void (`/dev/null`).

```bash
$ ls dir-that-doesnt-exist
ls: cannot access dir-that-doesnt-exist: No such file or directory # this is a stderr output`
```

```bash
$ ls ~
bin cache Desktop Downloads ... # this is a stdout output!
```

<div style='text-align: right'>Sbatch Scripting Features 8</div>

### Redirecting job's `stderr` and `stdout`
By default:
* job's working directory is the directory you submitted from
* `stdout` is directed to `slurm-<jobid>.out` in the job's working directory
* `stderr` is directed to wherever `stdout` is directed to

Redirect `stderr` and `stdout` with `--error` and `--output` options. They work with both relative and absolute paths, e.g.
```
--error=/dev/null
--output=path/to/output.out
```
where paths are resolved relative to the job's working directory.

Variables can be used, like:
* `%j`: job ID
* `%x`: job name
* `%u`: username
* `%t`: task ID i.e., seperate file per task
* `%N`: node name i.e., seperate file per nodes in job

<div style='text-align: right'>Sbatch Scripting Features 9</div>

```bash
#!/bin/bash
# Example sbatch script running Rscript
# Does a matmul
# rev2 - added --output and --error options

#SBATCH --mem=8G
#SBATCH --cpus-per-task=2
#SBATCH --time=1-
#SBATCH --nodes=1
#SBATCH --ntasks=1
#SBATCH --mail-user=yang.e@wehi.edu.au
#SBATCH --mail-type=ALL
#SBATCH --output=logs/matmul-%j.out
#SBATCH --error=logs-debug/matmul-%j.err

# loading module for R
module load R/openBLAS/4.2.1

Rscript matmul.rscript
```

<div style='text-align: right'>Sbatch scripting features 10</div>

### Using job dependancies
Slurm allows for submitted jobs to wait for another job to start or finish before beginning. While probably not as effective as workflow managers like Nextflow, Slurm's job dependencies can still be useful for simple workflows.

Make a job dependant on another by passing the `--dependency` option with one of the following values:
* `afterok:jobid1:jobid2...` waits for `jobid1`, `jobid2` ... to complete successfully
* `afterany:jobid1:jobid2...` "                              " to finish (fail, complete, cancelled)
* `after:jobid1:jobid2...` "                                 " to start or are cancelled.

e.g. `--dependency=afterok:12345678` will make the job wait for job `12345678` to complete successfully before starting.

<div style='text-align: right'>Sbatch scripting features 11</div>

### Job dependancies example: Recursive jobs
Recursive jobs are _one_ way to work with short QOS time limits.

Multiple Slurm jobs are submitted with a sequential dependancy pattern, i.e., the second job depends on the first, the third job depends on the second and so on...

Slurm script:

In [22]:
cat demo-scripts/recursive-job.sh

#!/bin/bash
#SBATCH --ntasks=1
#SBATCH --cpus-per-task=2
#SBATCH --mem-per-cpu=2G
#SBATCH --time=1
sleep 10


<div style='text-align: right'>Sbatch scripting features 12</div>
We could submit the Slurm script using a for loop:

In [20]:
# cell to run recursive script
prereq_jobid=$(sbatch --parsable demo-scripts/recursive-job.sh)
echo $prereq_jobid
for i in {1..5}; do
    prereq_jobid=$(sbatch --parsable --dependency=afterok:$prereq_jobid demo-scripts/recursive-job.sh)
    echo $prereq_jobid
done
squeue -u $USER

8703619
8703620
8703621
8703622
8703623
8703624
             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
           8703594 gpuq_larg interact   yang.e  R      21:18      1 gpu-a100-n01
           8701908 interacti sys/dash   yang.e  R    2:07:28      1 sml-n01
           8703624   regular test-rec   yang.e PD       0:00      1 (Dependency)
           8703623   regular test-rec   yang.e PD       0:00      1 (Dependency)
           8703622   regular test-rec   yang.e PD       0:00      1 (Dependency)
           8703621   regular test-rec   yang.e PD       0:00      1 (Dependency)
           8703620   regular test-rec   yang.e PD       0:00      1 (Dependency)
           8703619   regular test-rec   yang.e  R       0:00      1 sml-n05
           8703616   regular test-rec   yang.e  R       0:22      1 sml-n02


<div style='text-align: right'>Sbatch scripting features 13</div>
Breaking down the loop:

1. `prereq_jobid=$(sbatch --parsable demo-scripts/recursive-job.sh)`
    * Initial submit of recursive-job.sh
    * uses the `--parsable` option to get the job id from `sbatch`
    * the returned jobid is saved as `prereq_jobid`

2. `for i in {1..5}; do`
    * a bash `for` loop that loops through 1 to 5, where i is the looping variable

3. `prereq_jobid=$(sbatch --parsable --dependency=afterok:${prereq_jobid} demo-scripts/recursive-job.sh`
    * similar to 1.
    * adds the `--dependency=afterok:${prereq_jobid}` option to link jobs
    * `afterany` may be preferred instead of `afterok`
    * the newly submitted jobid overwrites the `prereq_jobid` variable

<div style='text-align: right'>Sbatch scripting features 14</div>

### Making use of job environments and interpreters
By default, when you submit a Slurm job, Slurm copies all the environment variables in your environment and adds some extra for the job to use.

In [26]:
export VAR1="here is some text"

In [30]:
cat demo-scripts/env-vars1.sbatch

#!/bin/bash

echo $VAR1


In [37]:
sbatch demo-scripts/env-vars1.sbatch

Submitted batch job 8681656


In [38]:
cat slurm-8681656.out

here is some text


<strong>Note</strong>: For reproducibility reasons, a Slurm script that relies on environment variables can be submitted inside a wrapper script which first exports the relevant variable.

<div style='text-align: right'>Sbatch scripting features 15</div>
Alternatively, you can use the `--export` option which allows you to set specific values

In [48]:
echo $VAR1

here is some text


In [49]:
sbatch --export=VAR1="this is some different text" demo-scripts/env-vars1.sbatch

Submitted batch job 8681761


In [50]:
cat slurm-8681761.out

this is some different text


This feature is especially useful when submitting jobs inside wrapper scripts.

You can also use the `--export-file` option to specify a file with a list of `VAR=value` pairs that you wish the script to use.

<div style='text-align: right'>Sbatch scripting features 16</div>
Slurm also adds environment variables that enable job parameters use

In [45]:
cat demo-scripts/env-vars2.sbatch

#!/bin/bash
#SBATCH --ntasks=1
#SBATCH --cpus-per-task=2

echo I am running on ${SLURM_NODELIST}
echo with ${SLURM_NTASKS} tasks
echo and ${SLURM_CPUS_PER_TASK} CPUs per task


In [46]:
sbatch demo-scripts/env-vars2.sbatch

Submitted batch job 8681710


In [47]:
cat slurm-8681710.out

I am running on sml-n03
with 1 tasks
and 2 CPUs per task


These Slurm environment variables make it easy to supply parallelisation parameters to a program e.g. specifying number of threads. 

<div style='text-align: right'>Sbatch scripting features 17</div>

### Submitting scripts with different interpreters
Typically scripts submitted by `sbatch` use the `bash` or `sh` interpreter (e.g. `#!/bin/bash`), but it may be more convenient to use a different interpreter.

You can do this by changing the "hash bang" statement at the top of the script. To demonstrate this, we can take our original R matmul script, and add a "hash bang" statement to the top.

In [52]:
cat demo-scripts/matmul-interpreter1.rscript | head -n 5

#!/usr/bin/env Rscript
## matmul.rscript

print("starting the matmul R script!")
nrows = 1e3


The statement in the above looks for the Rscript in your current environment. This statement only works because Slurm will copy your environment when a Slurm script is submitted.

`python` works similarly. Replace `Rscript` in the hash bang statement to `python`.

Alternatively, you can specify the absolute path to the interpreter.

e.g. `#!/stornext/System/data/apps/R/openBLAS/R-4.2.1/lib64/R/bin/Rscript`

<strong>Tip</strong>: you can use `--export=R_LIBS_USER=...` to point Rscript to your libraries (or `PYTHONPATH` for python)

<div style='text-align: right'>Sbatch scripting features 18</div>
Changing the interpreter still allows you to access the extra Slurm environment variables, but in a way appropriate to the interpreter.

R example:<br>
```r
slurmtasks <- Sys.getenv("SLURM_NTASKS")
```

Python example:
```python
import os
slurmtasks = os.getenv('SLURM_NTASKS')
```

Excercise:
Add a "using \<ntasks> tasks and \<cpus-per-task> CPUs per task" statement to the matmul R script. Submit the script to confirm it works.

The output should look something like:
```
[1] "starting the matmul R script!"
[1] "using 1 tasks and 2 CPUs per task"
[1] "elem: 1000*1000 = 1e+06"
Time difference of 0.06340098 secs
```

## Handling Embarrasingly Parallel Workflows
Making life easier with job arrays

<div style="text-align: right">Handling Embarrasingly Parallel Workflows 1</div>

### Embarrasingly Parallel jobs
Embarrasingly parallel computation is computation that can occur in parallel with minimal coordination. This type of parallel computation is _very_ common.

Examples are parameter scans, genomic sequencing, basecalling, folding@home ...

Embarassingly parallel problems are facilitated in Slurm by "array jobs". Array jobs allows you to use a single script to submit multiple jobs with similar functionality.

The main benefits to using an array job are:
* they are easier to program
    * each array job is grouped under a single job ID and distinguished by array indices e.g. 1234567_1, 1234567_2, ...
    * no need for wrapper scripts/loops
    * can make use of Slurm environment variables to coordinate

* Slurm handles large array jobs better than many individual jobs
    * helps the Slurm process the queue faster

<div style="text-align: right">Handling Embarrasingly Parallel Workflows 2</div>

### Setting up an array job
Array jobs are created by adding the `--array=start-end` option. Slurm jobs, AKA "tasks", will be created with indices between `start` and `end`. e.g. `--array=1-10` will create tasks with indices 1, 2, ..., 10.

`start` and `end` values can be within 0 and 1000 (inclusive). Note this is site specific.

Singular values or discrete lists can also be specific e.g. `--array=1` or `--array=1,3,5,7-10`.

In [2]:
cat demo-scripts/matmul-array1.rscript | head

#!/usr/bin/env Rscript
#SBATCH --ntasks=1
#SBATCH --cpus-per-task=2
#SBATCH --mem=4G
#SBATCH --array=1-10

## matmul.rscript

print("starting the matmul R script!")
paste("using", Sys.getenv("SLURM_NTASKS"), "tasks")


<div style="text-align: right">Handling Embarrasingly Parallel Workflows 3</div>

### Using array indices in other SBATCH output and error
Slurm augments the default output behaviour of array jobs automatically.

If no `--output` option is provided, an array job will produce a an output file `slurm-<jobid>-<arrayindex>.out` for each index in the array.

If you specify `--output` and `--error`, then you can use `%A` and `%a` variables, which represent the job index and the array index, respectively.

e.g.

In [1]:
cat demo-scripts/matmul-array2.rscript | head

#!/usr/bin/env Rscript
#SBATCH --ntasks=1
#SBATCH --cpus-per-task=2
#SBATCH --mem=4G
#SBATCH --array=1-10
#SBATCH --output=Rmatmul-times-%A-%a.out
#SBATCH --error=Rmatmul-times-%A-%a.err

## matmul.rscript



<div style="text-align: right">Handling Embarrasingly Parallel Workflows 4</div>

### Using array indices in script body
Each task in the array can make use of its index to enable parallelism. This is by making use of the `SLURM_ARRAY_TASK_ID` environment variable.

Other environment variables are accessible:
* `SLURM_ARRAY_JOB_ID` the job Id of the entire job array
* `SLURM_ARRAY_TASK_COUNT` the _number_ of tasks in the array
* `SLURM_ARRAY_TASK_MAX` the largest ID of tasks in the array
* `SLURM_ARRAY_TASK_MIN` the smallest ID of tasks in the array

Exercise: Add a paste statement to the matmul R script that prints the task ID

The output of each job task should look something like:
```
[1] "starting the matmul R script!"
[1] "using 1 tasks and 2 CPUs per task"
[1] "I am job task 1 in an array of 10!"
[1] "elem: 1000*1000 = 1e+06"
Time difference of 0.06340098 secs
```

```r
#!/usr/bin/env Rscript
#SBATCH --ntasks=1
#SBATCH --cpus-per-task=2
#SBATCH --mem=4G
#SBATCH --array=1-10
#SBATCH --output=Rmatmul-times-%A-%a.out
#SBATCH --error=Rmatmul-times-%A-%a.err

## matmul.rscript

print("starting the matmul R script!")

paste("using", Sys.getenv("SLURM_NTASKS"), 
      "tasks and", 
      Sys.getenv("SLURM_CPUS_PER_TASK"), 
      "CPUs per task")

paste("I am job task", 
      Sys.getenv("SLURM_ARRAY_TASK_ID"), 
      "in an array of", 
      Sys.getenv("SLURM_ARRAY_TASK_COUNT"))

nrows = 1e3
print(paste0("elem: ", nrows, "*", nrows, " = ", nrows*nrows))

# generating matrices
M <- matrix(rnorm(nrows*nrows),nrow=nrows)
N <- matrix(rnorm(nrows*nrows),nrow=nrows)

# start matmul
start.time <- Sys.time()
invisible(M %*% N)
end.time <- Sys.time()

# Getting final time and writing to stdout
elapsed.time <- difftime(time1=end.time, time2=start.time, units="secs")
print(elapsed.time)
```

<div style="text-align: right">Handling Embarrasingly Parallel Workflows 5</div>
Excercise: modify the `nrows` variable to equal 10*taskID 

hint: you will need the `strtoi` function

Your output from job task 1 should look like:
```
[1] "starting the matmul R script!"
[1] "using 1 tasks and 2 CPUs per task"
[1] "I am job task 1 in an array of 10!"
[1] "elem: 10*10 = 100"
Time difference of 0.06340098 secs
```

```r
#!/usr/bin/env Rscript
#SBATCH --ntasks=1
#SBATCH --cpus-per-task=2
#SBATCH --mem=4G
#SBATCH --array=1-10
#SBATCH --output=Rmatmul-times-%A-%a.out
#SBATCH --error=Rmatmul-times-%A-%a.err

## matmul.rscript

print("starting the matmul R script!")

paste("using", Sys.getenv("SLURM_NTASKS"), 
      "tasks and", 
      Sys.getenv("SLURM_CPUS_PER_TASK"), 
      "CPUs per task")

paste("I am job task", 
      Sys.getenv("SLURM_ARRAY_TASK_ID"), 
      "in an array of", 
      Sys.getenv("SLURM_ARRAY_TASK_COUNT"))

#nrows = 1e3
nrows <- 10 * strtoi(Sys.getenv("SLURM_ARRAY_TASK_ID"))

print(paste0("elem: ", nrows, "*", nrows, " = ", nrows*nrows))

# generating matrices
M <- matrix(rnorm(nrows*nrows),nrow=nrows)
N <- matrix(rnorm(nrows*nrows),nrow=nrows)

# start matmul
start.time <- Sys.time()
invisible(M %*% N)
end.time <- Sys.time()

# Getting final time and writing to stdout
elapsed.time <- difftime(time1=end.time, time2=start.time, units="secs")
print(elapsed.time)
```

<div style="text-align: right">Handling Embarrasingly Parallel Workflows 6</div>

For workflows requiring input files or parameters, there are multiple ways you can use job arrays:
* folders with job array indices with the necessary input files
* programmatically generate CSV file with parameters or paths to files where row/column no. correspond to task IDs
* if/else or select case statements in the script
* ...

<div style="text-align: right">Handling Embarrasingly Parallel Workflows 7</div>

### Job arrays and job dependencies
What you can't do:
* make job array tasks depend on one another

What you can do:
* make jobs dependent on entire job arrays
    * `--dependency=afterok:<jobid>`
* make job arrays dependent on other job/job arrays
* make jobs dependent on job array tasks
    * `--dependency=afterok:<jobid>_<taskid>`

## Conclusion

Thanks for attending WEHI's first intermediate Slurm workshop!

<div style="text-align: right">Conclusion 1</div>

Please fill out our feedback form:

<a href="https://forms.office.com/pages/responsepage.aspx?id=4nJ8qs_FQEqiKqWFwEygfJ2o16JnCyxPmA6-Ggzd8tNUMEpQUEZGUVRPWFEyNzhSUzE1VDRQQlUyRy4u">https://forms.office.com/pages/responsepage.aspx?id=4nJ8qs_FQEqiKqWFwEygfJ2o16JnCyxPmA6-Ggzd8tNUMEpQUEZGUVRPWFEyNzhSUzE1VDRQQlUyRy4u</a>

We use these forms to help decide which workshops to run in the future and improve our current workshops!


Contact us at research.computing@wehi.edu.au for any kind of help related to computing and research!